In [1]:
import csv
from Bio import SeqIO
from collections import Counter
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [2]:
# List of amino acids and all possible dipeptides
amino_acids = "ARNDCEQGHILKMFPSTWYV"
dipeptides = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]

def calculate_amino_acid_percentage(sequence):
    """Calculate the percentage of each amino acid in the given sequence."""
    total_length = len(sequence)
    amino_acid_counts = Counter(sequence)
    return {aa: (amino_acid_counts[aa] / total_length) * 100 if total_length > 0 else 0 for aa in amino_acids}

'''
def calculate_dipeptide_percentage(sequence):
    """Calculate the percentage of each dipeptide in the given sequence."""
    dipeptide_list = [sequence[i:i+2] for i in range(len(sequence) - 1)]
    dipeptide_counts = Counter(dipeptide_list)
    total_dipeptides = len(dipeptide_list)
    return {dp: (dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides}
'''

def calculate_dipeptide_percentage(sequence, g):
    """Calculate the percentage of each dipeptide in the given sequence."""
    dipeptide_list = [sequence[i]+sequence[i+g] for i in range(len(sequence) - g)]
    dipeptide_counts = Counter(dipeptide_list)
    total_dipeptides = len(dipeptide_list)
    return {dp: (dipeptide_counts[dp] / total_dipeptides) * 100 if total_dipeptides > 0 else 0 for dp in dipeptides}

def aliphatic_index(sequence):
    analysed_seq = ProteinAnalysis(sequence)
    aa_counts = analysed_seq.count_amino_acids()
    
    A = aa_counts['A']  # Alanine
    V = aa_counts['V']  # Valine
    I = aa_counts['I']  # Isoleucine
    L = aa_counts['L']  # Leucine
    seq_length = sum(aa_counts.values())  # Total amino acids
    
    # Aliphatic index formula
    AI = ((A + 2.9 * V + 3.9 * (I + L)) / seq_length) * 100
    return AI

In [3]:
# 6-CTD
group1 = {
    'hydrophobicity_PRAM900101': 'RKEDQN',
    'hydrophobicity_ARGP820101': 'QSTNGDE',
    'hydrophobicity_ZIMJ680101': 'QNGSWTDERA',
    'hydrophobicity_PONP930101': 'KPDESNQT',
    'hydrophobicity_CASG920101': 'KDEQPSRNTG',
    'hydrophobicity_ENGD860101': 'RDKENQHYP',
    'hydrophobicity_FASG890101': 'KERSQD',
    'normwaalsvolume': 'GASTPDC',
    'polarity':        'LIFWCMVY',
    'polarizability':  'GASDT',
    'charge':          'KR',
    'secondarystruct': 'EALMQKRH',
    'solventaccess':   'ALFCGIVW'
}
group2 = {
    'hydrophobicity_PRAM900101': 'GASTPHY',
    'hydrophobicity_ARGP820101': 'RAHCKMV',
    'hydrophobicity_ZIMJ680101': 'HMCKV',
    'hydrophobicity_PONP930101': 'GRHA',
    'hydrophobicity_CASG920101': 'AHYMLV',
    'hydrophobicity_ENGD860101': 'SGTAW',
    'hydrophobicity_FASG890101': 'NTPG',
    'normwaalsvolume': 'NVEQIL',
    'polarity':        'PATGS',
    'polarizability':  'CPNVEQIL',
    'charge':          'ANCQGHILMFPSTWYV',
    'secondarystruct': 'VIYCWFT',
    'solventaccess':   'RKQEND'
}
group3 = {
    'hydrophobicity_PRAM900101': 'CLVIMFW',
    'hydrophobicity_ARGP820101': 'LYPFIW',
    'hydrophobicity_ZIMJ680101': 'LPFYI',
    'hydrophobicity_PONP930101': 'YMFWLCVI',
    'hydrophobicity_CASG920101': 'FIWC',
    'hydrophobicity_ENGD860101': 'CVLIMF',
    'hydrophobicity_FASG890101': 'AYHWVMFLIC',
    'normwaalsvolume': 'MHKFRYW',
    'polarity':        'HQRKNED',
    'polarizability':  'KMHFRYW',
    'charge':          'DE',
    'secondarystruct': 'GNPSD',
    'solventaccess':   'MSPTHY'
}
groups = [group1, group2, group3]
ctd_property = (
'hydrophobicity_PRAM900101', 'hydrophobicity_ARGP820101', 'hydrophobicity_ZIMJ680101', 'hydrophobicity_PONP930101',
'hydrophobicity_CASG920101', 'hydrophobicity_ENGD860101', 'hydrophobicity_FASG890101', 'normwaalsvolume',
'polarity', 'polarizability', 'charge', 'secondarystruct', 'solventaccess')
def ctd_composition(sequence):
    c = Counter(sequence)
    encodings = []
    for p in ctd_property:
        c1 = sum([c[k] for k in group1[p]])
        c2 = sum([c[k] for k in group2[p]])
        encodings += [c1, c2]
    return encodings
    '''
    def Count(seq1, seq2):
        s = 0
        for aa in seq1:
            s += seq2.count(aa)
        return s
    encodings = []
    for i in sequence:
        for p in ctd_property:
            c1 = Count(group1[p], sequence) / len(sequence)
            c2 = Count(group2[p], sequence) / len(sequence)
            c3 = 1 - c1 - c2
            encodings = encodings + [c1, c2, c3]
    return encodings
    '''
transitions = {}
total_transitions = {}
for p in ctd_property:
    transitions[p] = {'group1-group2': 0, 'group1-group3': 0, 'group2-group1': 0, 'group2-group3': 0, 'group3-group1': 0, 'group3-group2': 0}
    total_transitions[p] = 0
def ctd_transitions(sequence):
    for i in range(len(sequence)-1):
        a1, a2 = sequence[i], sequence[i+1]
        for p in ctd_property:
            if a1 in group1[p] and a2 in group2[p]: transitions[p]['group1-group2'] += 1
            if a1 in group1[p] and a2 in group3[p]: transitions[p]['group1-group3'] += 1
            if a1 in group2[p] and a2 in group1[p]: transitions[p]['group2-group1'] += 1
            if a1 in group2[p] and a2 in group3[p]: transitions[p]['group2-group3'] += 1
            if a1 in group3[p] and a2 in group1[p]: transitions[p]['group3-group1'] += 1
            if a1 in group3[p] and a2 in group2[p]: transitions[p]['group3-group2'] += 1
    for p in ctd_property:
        total_transitions[p] = sum(transitions[p].values())
        #for k in transitions[p]:
        #    total_transitions[p] += transitions[p][k]
    #total_transitions = sum(transition.values())
    #if total_transitions == 0: return transitions
    return {p+'.'+k: (v / total_transitions[p])*100 if total_transitions[p] != 0 else 0 for p in ctd_property for k, v in transitions[p].items()}
 


In [4]:
def calculate_protein_features(sequence):
    """Calculate protein features using Biopython's ProteinAnalysis."""
    analyzed_seq = ProteinAnalysis(str(sequence))
    AI = aliphatic_index(sequence)

    return {
        "Length": len(sequence),
        #"Molecular Weight": analyzed_seq.molecular_weight(),
        #"Aromaticity": analyzed_seq.aromaticity(),
        #"GRAVY": analyzed_seq.gravy(),
        #"Isoelectric Point": analyzed_seq.isoelectric_point(),
        #"Helix Fraction": analyzed_seq.secondary_structure_fraction()[0],
        #"Turn Fraction": analyzed_seq.secondary_structure_fraction()[1],
        #"Sheet Fraction": analyzed_seq.secondary_structure_fraction()[2],
        #"hydrophobicity": sum(analyzed_seq.protein_scale(window=5, param_dict=KyteDoolittle1)) / len(sequence),
        #"instability_index": analyzed_seq.instability_index(),
        #"extenictioin_coeff_X": analyzed_seq.molar_extinction_coefficient()[0],
        #"extenictioin_coeff_Y": analyzed_seq.molar_extinction_coefficient()[1],
        ##"aa_composition": aa_composition = analysed_seq.count_amino_acids(),
        #"aliphatic_index": AI,
    }
    

In [5]:
maxGap = 3
def process_proteomes(proteome_files, proteome_label, output_csv):
    """Process sequences from each proteome file and append features to CSV."""
    with open(output_csv, "a", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)

        for file_path in proteome_files:
            print(file_path)
            for seq_record in SeqIO.parse(file_path, "fasta"):
                sequence = str(seq_record.seq)

                # **Skip sequences that contain 'X'**
                #if "X" in sequence:
                #    continue  # Skip this sequence entirely
                #if "U" in sequence: 
                    #u_list.append(seq_record.seq)
                #    continue
                #if "n" in sequence: 
                    #n_list.append((seq_record.id, file_path))
                #    continue
                # Process valid sequences
                protein_features = calculate_protein_features(sequence)
                #aa_percentages = calculate_amino_acid_percentage(sequence)
                '''
                dipeptide_percentages = {}
                for g in range(1, maxGap):
                    dipeptide_percentages[g] = calculate_dipeptide_percentage(sequence, g)
                '''
                ctd_composition_percentages = ctd_composition(sequence)
                ctd_transitions_percentages = ctd_transitions(sequence)
                
                # Combine all features
                row = [
                    seq_record.id,
                    sequence,
                    proteome_label,
                    protein_features["Length"],
                    #protein_features["Molecular Weight"],
                    #protein_features["Aromaticity"],
                    #protein_features["GRAVY"],
                    #protein_features["Isoelectric Point"],
                    #protein_features["Helix Fraction"],
                    #protein_features["Turn Fraction"],
                    #protein_features["Sheet Fraction"],
                    #protein_features["hydrophobicity"],
                    #protein_features["instability_index"],
                    #protein_features["extenictioin_coeff_X"],
                    #protein_features["extenictioin_coeff_Y"],
                    #protein_features["aliphatic_index"],
                ]
                #row += [aa_percentages[aa] for aa in amino_acids]
                row += ctd_composition_percentages
                row += list(ctd_transitions_percentages.values())
                '''
                for g in range(1, maxGap):
                    row + [dipeptide_percentages[g][dp] for dp in dipeptides]
                '''
                csvwriter.writerow(row)

In [6]:
# File lists for both types of proteomes
thermophilic_files = [
    "A_aeolicus.txt", "C_abyssi.txt", "D_thermophilum.txt", 
    "P_methylaliphatogenes.txt", "R_marinus.txt", "S_thermophila.txt",
    "T_ islandicus.txt", "T_aquaticus.txt", "T_indicus.txt",
    "T_marianensis.txt", "T_maritima.txt", "T_velox.txt"
]

psychrophilic_files = [
    "A_sublithincola.txt", "C_gasigenes.txt", "D_frigens.txt",
    "G_pallidula.txt", "G_superstes.txt", "H_roseosalivarius.txt",
    "M_psychrophila.txt", "P_agardhii.txt", "P_atlanticus.txt", "T_saanensis.txt"
]

mesophilic_files = [
    "brachy_murdo.txt", "c_falvigena.txt", "deino_mari.txt", "ecoli.txt", 
    "emp_brevis.txt", "flecto_major.txt", "hol_foetida.txt", "keton_race.txt",
    "lepto_shahi.txt", "mycoplasma.txt", "pro_sphenisci_proteom.txt"
]

# Output CSV file
output_csv = "filtered_proteome_data5.csv"

# Write the header row to the combined CSV
with open(output_csv, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    header = [
        "Sequence ID", "Seq", "Proteome Type (0=Hot, 1=Cold, 2=Meso)", "Length", 
        #"Molecular Weight", 
        #"Aromaticity", "GRAVY", "Isoelectric Point", "Helix Fraction", "Turn Fraction", "Sheet Fraction",
        #"hydrophobicity","instability_index", "extenictioin_coeff_X", "extenictioin_coeff_Y", "aliphatic_index",
    ]
    #header += list(amino_acids) #+ dipeptides
    ## ctd_composition header
    
    for p in ctd_property:
        for k in range(1, 3):
            header.append(p + '.G' + str(k))
    
    # ctd_transition header
    for p in ctd_property:
        for k in transitions[p]:
            header.append(p+'.'+k)
    '''
    for p in range(maxGap):
        for k in dipeptides:
            header.append(k + str(p))
    '''
    print(len(header))
    csvwriter.writerow(header)

print('Processing thermophilic proteomes label 0')
# Process thermophilic proteomes (label = 0)
process_proteomes(thermophilic_files, 0, output_csv)

print('Processing psychrophilic proteomes label 1')
# Process psychrophilic proteomes (label = 1)
process_proteomes(psychrophilic_files, 1, output_csv)

# Process mesophilic proteomes (label = 2)
print('Processing mesophilic proteomes (label = 2)')
process_proteomes(mesophilic_files, 2, output_csv)

print(f"Amino acid, dipeptide, and structural features saved to {output_csv}.")

108
Processing thermophilic proteomes label 0
A_aeolicus.txt
C_abyssi.txt
D_thermophilum.txt
P_methylaliphatogenes.txt
R_marinus.txt
S_thermophila.txt
T_ islandicus.txt
T_aquaticus.txt
T_indicus.txt
T_marianensis.txt
T_maritima.txt
T_velox.txt
Processing psychrophilic proteomes label 1
A_sublithincola.txt
C_gasigenes.txt
D_frigens.txt
G_pallidula.txt
G_superstes.txt
H_roseosalivarius.txt
M_psychrophila.txt
P_agardhii.txt
P_atlanticus.txt
T_saanensis.txt
Processing mesophilic proteomes (label = 2)
brachy_murdo.txt
c_falvigena.txt
deino_mari.txt
ecoli.txt
emp_brevis.txt
flecto_major.txt
hol_foetida.txt
keton_race.txt
lepto_shahi.txt
mycoplasma.txt
pro_sphenisci_proteom.txt
Amino acid, dipeptide, and structural features saved to filtered_proteome_data5.csv.
